In [ ]:
# O código que segue lê as planilhas com os dados informados pelas coordeções dos Componentes Básico, Estratégico e Especializado
# E realiza carga na DBMOBILE - Tabelas: tb_medsus_droga, tb_medsus_categoria, rl_medsus_droga_categoria, rl_medsus_droga_campo, tb_medsus_campo, tb_medsus_tipo_campo e tb_medsus_tipo_categoria
# Autor: Jessé da Nóbrega Batista Azevêdo
# Data: 18/01/2024

In [34]:
# Importando bibliotecas
import pandas as pd

# Ler o arquivo xlsx
xlsx = pd.read_excel('../DATASET/dados_medsus_v18012023.xlsx', sheet_name=None, engine='openpyxl')

# Dicionário para armazenar os DataFrames
dfs = {}

for sheet_name, df in xlsx.items():
    dfs[sheet_name] = df

# Criando datasets para cada tipo componente da AF e Farmácia
df_basico = dfs['basica']
df_estrategico = dfs['farmacia']
df_especializado = dfs['estrategico']
df_farmacia = dfs['especializado']

In [3]:
# Criando conexão com database Oracle
import cx_Oracle

dsn_tns = cx_Oracle.makedsn('localhost', '1521', service_name='XE') 
conn = cx_Oracle.connect(user='C##DBMOBILE', password='abc123', dsn=dsn_tns)

In [6]:
#Tratando o dataset básico

# 1. Verificando integridade dos dados lidos
num_linhas, num_colunas = df_basico.shape
print(f'Número de linhas: {num_linhas}')
print(f'Número de colunas: {num_colunas}')

# 2. Separando apenas os títulos
titulos = df_basico.columns
print(titulos)

# 3. Dando o insert para nos títulos

cursor = conn.cursor()

# Iterando sobre os títulos
for i, titulo in enumerate(titulos, start=1):
    # Escrevendo a instrução SQL no arquivo
    query = f"INSERT INTO C##DBMOBILE.TB_MEDSUS_CAMPO (CO_SEQ_CAMPO, DS_CAMPO, CO_TIPO_CAMPO, ST_OBRIGATORIO, ST_PADRAO) VALUES ({i},'{titulo}', 1, 'N', 'N')"
    cursor.execute(query)

conn.commit()

Número de linhas: 410
Número de colunas: 8
Index(['CATMAT', 'DENOMINAÇÃO GENÉRICA', 'FORMA FARMACÊUTICA', 'GRUPO',
       'CID ou DOENÇA', 'DOCUMENTOS NECESSÁRIOS', 'COMO ACESSAR - REGRAS ',
       'ONDE ENCONTRAR'],
      dtype='object')


In [12]:
# 4. Criando o insert para os medicamentos
cursor = conn.cursor()

for i, row in df_basico.iterrows():
    query = f"INSERT INTO C##DBMOBILE.TB_MEDSUS_DROGA (CO_SEQ_DROGA, NO_DROGA, DT_ATUALIZADO, DT_CRIADO, ST_REGISTRO_ATIVO, CO_UTILIZADOR, ST_PUBLICADO, DS_JUSTIFICATIVA) VALUES( {i}, '{row['DENOMINAÇÃO GENÉRICA']}', CURRENT_TIMESTAMP , CURRENT_TIMESTAMP , 'S' , 1541, 'S' , NULL )"
    cursor.execute(query)

conn.commit()

In [16]:
# 5. Adicionando fonte de financiamento
cursor = conn.cursor()
consulta = f"select * from C##DBMOBILE.TB_MEDSUS_DROGA"
cursor.execute(consulta)
rows = cursor.fetchall()

for row in rows:
    query = f"INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CATEGORIA (CO_SEQ_DROGA_CATEGORIA, CO_DROGA, CO_CATEGORIA, DT_CRIADO, ST_REGISTRO_ATIVO, CO_UTILIZADOR) VALUES( SQ_MEDDROGACAT_COSEQDROGACAT.NEXTVAL, {row[0]}, 1, CURRENT_TIMESTAMP, 'S', 1541)"
    cursor.execute(query)

conn.commit()

In [29]:
df_basico.head()

,CATMAT,DENOMINAÇÃO GENÉRICA,FORMA FARMACÊUTICA,GRUPO,CID ou DOENÇA,DOCUMENTOS NECESSÁRIOS,COMO ACESSAR - REGRAS,ONDE ENCONTRAR
0,BR0270597,ACETATO DE BETAMETASONA 3 mg/mL + FOSFATO DISS...,SUSPENSÃO INJETÁVEL,NaN,NaN,É necessário apresentar:\n- Receita médica;\n-...,NaN,O usuário poderá acessar o medicamento nas uni...
1,BR0345240,ACETATO DE HIDROCORTISONA 10 mg/g,POMADA,NaN,NaN,É necessário apresentar:\n- Receita médica;\n-...,NaN,O usuário poderá acessar o medicamento nas uni...
2,BR0416731,ACETATO DE MEDROXIPROGESTERONA + CIPIONATO DE ...,SUSPENSÃO INJETÁVEL,NaN,NaN,É necessário apresentar:\n- Receita médica;\n-...,NaN,O usuário poderá acessar o medicamento nas uni...
3,BR0271445,ACETATO DE MEDROXIPROGESTERONA 10 mg,COMPRIMIDO,NaN,NaN,É necessário apresentar:\n- Receita médica;\n-...,NaN,O usuário poderá acessar o medicamento nas uni...
4,BR0398702U0005,ACETATO DE MEDROXIPROGESTERONA 150 mg/mL,SUSPENSÃO INJETÁVEL,NaN,NaN,É necessário apresentar:\n- Receita médica;\n-...,NaN,O usuário poderá acessar o medicamento nas uni...


In [38]:
# 6. Categoria campo
import math
import traceback
cursor = conn.cursor()

for i, row in df_basico.iterrows():
    try:
        CO_CONSULTA = f"SELECT CO_SEQ_DROGA FROM C##DBMOBILE.TB_MEDSUS_DROGA WHERE NO_DROGA = '{row['DENOMINAÇÃO GENÉRICA']}'"
        cursor.execute(CO_CONSULTA)
        resultado = cursor.fetchone()
        resultado = int(resultado[0])

        # FORMA FARMACEUTICA
        forma_farmaceutica = row['FORMA FARMACÊUTICA'].replace("'", "''")

        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 3"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0:
            query01 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 3, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 1, 'S', '{forma_farmaceutica}')"""
            cursor.execute(query01)

        #GRUPO
        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 4"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0 and not math.isnan(row['GRUPO']):
            query02 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 4, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 2, 'S', '{row['GRUPO']}')"""
            cursor.execute(query02)
        # if math.isnan(row['GRUPO']):
        #     query02 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 4, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 2, 'S', '{row['GRUPO']}')"""
        #     cursor.execute(query02)

        #CID ou DOENÇA
        if not math.isnan(row['CID ou DOENÇA']):
            lista_original07 = str(row['DOCUMENTOS NECESSÁRIOS']).split('\n')
            lista_alterada07 = ["<p>" + texto + "</p>" for texto in lista_original07]
            texto_final07 = ''.join(lista_alterada07)
            
            query03 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 5, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 3, 'S', '{texto_final07}')"""
            cursor.execute(query03)

        # Documentos Necessários
        lista_original = str(row['DOCUMENTOS NECESSÁRIOS']).split('\n')
        lista_alterada = ["<p>" + texto + "</p>" for texto in lista_original]
        texto_final = ''.join(lista_alterada)

        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 6"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0:
            query04 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 6, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 4, 'S', '{texto_final}')"""
            cursor.execute(query04)

        #COMO ACESSAR
        if not math.isnan(row['COMO ACESSAR']):
            lista_original01 = str(row['COMO ACESSAR']).split('\n')
            lista_alterada01 = ["<p>" + texto + "</p>" for texto in lista_original01]
            texto_final01 = ''.join(lista_alterada01)

            query05 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 7, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 5, 'S', '{texto_final01}')"""
            cursor.execute(query05)

        #ONDE ENCONTRAR
        if row['ONDE ENCONTRAR']:
            lista_original02 = str(row['ONDE ENCONTRAR']).split('\n')
            lista_alterada02 = ["<p>" + texto + "</p>" for texto in lista_original02]
            texto_final02 = ''.join(lista_alterada02)

            query06 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 8, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 6, 'S', '{texto_final02}')"""
            cursor.execute(query06)       

        conn.commit()
    except Exception as e:
        print("Ocorreu um erro: ", e)
        print(traceback.format_exc())
        conn.rollback()

Ocorreu um erro:  ORA-00001: unique constraint (C##DBMOBILE.UK_MEDDROGACAM_CODROGA_COCAMPO) violated
Traceback (most recent call last):
  File "/tmp/ipykernel_75060/4021357574.py", line 74, in <module>
    cursor.execute(query06)
cx_Oracle.IntegrityError: ORA-00001: unique constraint (C##DBMOBILE.UK_MEDDROGACAM_CODROGA_COCAMPO) violated

Ocorreu um erro:  ORA-00001: unique constraint (C##DBMOBILE.UK_MEDDROGACAM_CODROGA_COCAMPO) violated
Traceback (most recent call last):
  File "/tmp/ipykernel_75060/4021357574.py", line 74, in <module>
    cursor.execute(query06)
cx_Oracle.IntegrityError: ORA-00001: unique constraint (C##DBMOBILE.UK_MEDDROGACAM_CODROGA_COCAMPO) violated

Ocorreu um erro:  ORA-00001: unique constraint (C##DBMOBILE.UK_MEDDROGACAM_CODROGA_COCAMPO) violated
Traceback (most recent call last):
  File "/tmp/ipykernel_75060/4021357574.py", line 74, in <module>
    cursor.execute(query06)
cx_Oracle.IntegrityError: ORA-00001: unique constraint (C##DBMOBILE.UK_MEDDROGACAM_CODROGA

In [ ]:
# FARMACIA POPULARE

In [43]:
# 4. Criando o insert para os medicamentos
cursor = conn.cursor()

for i, row in df_farmacia.iterrows():
    CO_CONSULTA = f"SELECT CO_SEQ_DROGA FROM C##DBMOBILE.TB_MEDSUS_DROGA WHERE NO_DROGA = '{row['DENOMINAÇÃO GENÉRICA']}'"
    cursor.execute(CO_CONSULTA)
    resultado = cursor.fetchone()

    if resultado is not None:
        resultado = int(resultado[0])

        # Verificar se a combinação de CO_DROGA e CO_CATEGORIA já existe
        query_check = f"SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CATEGORIA WHERE CO_DROGA = {resultado} AND CO_CATEGORIA = 33"
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        # Se a combinação não existir, inserir
        if count == 0:
            query = f"INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CATEGORIA (CO_SEQ_DROGA_CATEGORIA, CO_DROGA, CO_CATEGORIA, DT_CRIADO, ST_REGISTRO_ATIVO, CO_UTILIZADOR) VALUES( SQ_MEDDROGACAT_COSEQDROGACAT.NEXTVAL, {resultado}, 33, CURRENT_TIMESTAMP, 'S', 1541)"
            cursor.execute(query)
    else:
        # Obter o valor máximo de CO_SEQ_DROGA
        query_max = "SELECT MAX(CO_SEQ_DROGA) FROM C##DBMOBILE.TB_MEDSUS_DROGA"
        cursor.execute(query_max)
        max_value = cursor.fetchone()[0]

        # Adicionar 1 ao valor máximo
        new_value = max_value + 1

        # Usar o novo valor na consulta de inserção
        query = f"INSERT INTO C##DBMOBILE.TB_MEDSUS_DROGA (CO_SEQ_DROGA, NO_DROGA, DT_ATUALIZADO, DT_CRIADO, ST_REGISTRO_ATIVO, CO_UTILIZADOR, ST_PUBLICADO, DS_JUSTIFICATIVA) VALUES( {new_value}, '{row['DENOMINAÇÃO GENÉRICA']}', CURRENT_TIMESTAMP , CURRENT_TIMESTAMP , 'S' , 1541, 'S' , NULL )"
        cursor.execute(query)

        query = f"INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CATEGORIA (CO_SEQ_DROGA_CATEGORIA, CO_DROGA, CO_CATEGORIA, DT_CRIADO, ST_REGISTRO_ATIVO, CO_UTILIZADOR) VALUES( SQ_MEDDROGACAT_COSEQDROGACAT.NEXTVAL, {new_value}, 33, CURRENT_TIMESTAMP, 'S', 1541)"
        cursor.execute(query)

conn.commit()

In [44]:
# 6. Categoria campo
import math
import traceback
cursor = conn.cursor()

for i, row in df_farmacia.iterrows():
    try:
        CO_CONSULTA = f"SELECT CO_SEQ_DROGA FROM C##DBMOBILE.TB_MEDSUS_DROGA WHERE NO_DROGA = '{row['DENOMINAÇÃO GENÉRICA']}'"
        cursor.execute(CO_CONSULTA)
        resultado = cursor.fetchone()
        resultado = int(resultado[0])

        # FORMA FARMACEUTICA
        forma_farmaceutica = row['FORMA FARMACÊUTICA'].replace("'", "''")

        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 3"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0:
            query01 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 3, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 1, 'S', '{forma_farmaceutica}')"""
            cursor.execute(query01)

        #GRUPO
        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 4"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0 and not math.isnan(row['GRUPO']):
            query02 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 4, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 2, 'S', '{row['GRUPO']}')"""
            cursor.execute(query02)
        # if math.isnan(row['GRUPO']):
        #     query02 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 4, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 2, 'S', '{row['GRUPO']}')"""
        #     cursor.execute(query02)

        #CID ou DOENÇA
        if not math.isnan(row['CID ou DOENÇA']):
            lista_original07 = str(row['DOCUMENTOS NECESSÁRIOS']).split('\n')
            lista_alterada07 = ["<p>" + texto + "</p>" for texto in lista_original07]
            texto_final07 = ''.join(lista_alterada07)
            
            query03 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 5, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 3, 'S', '{texto_final07}')"""
            cursor.execute(query03)

        # Documentos Necessários
        lista_original = str(row['DOCUMENTOS NECESSÁRIOS']).split('\n')
        lista_alterada = ["<p>" + texto + "</p>" for texto in lista_original]
        texto_final = ''.join(lista_alterada)

        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 6"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0:
            query04 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 6, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 4, 'S', '{texto_final}')"""
            cursor.execute(query04)

        #COMO ACESSAR
        if not math.isnan(row['COMO ACESSAR']):
            lista_original01 = str(row['COMO ACESSAR']).split('\n')
            lista_alterada01 = ["<p>" + texto + "</p>" for texto in lista_original01]
            texto_final01 = ''.join(lista_alterada01)

            query05 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 7, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 5, 'S', '{texto_final01}')"""
            cursor.execute(query05)

        #ONDE ENCONTRAR
        if row['ONDE ENCONTRAR']:
            lista_original02 = str(row['ONDE ENCONTRAR']).split('\n')
            lista_alterada02 = ["<p>" + texto + "</p>" for texto in lista_original02]
            texto_final02 = ''.join(lista_alterada02)

            query06 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 8, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 6, 'S', '{texto_final02}')"""
            cursor.execute(query06)       

        conn.commit()
    except Exception as e:
        print("Ocorreu um erro: ", e)
        print(traceback.format_exc())
        conn.rollback()

Ocorreu um erro:  'GRUPO'
Traceback (most recent call last):
  File "/data/Trabalho/MEDSUS/medsus/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3791, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 152, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 181, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'GRUPO'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/ipykernel_75060/3098021767.py", line 29, in <module>
    if count == 0 and not math.isnan(row['GRUPO']):
  File "/data/Trabalho/MEDSUS/medsus/lib/python3.9/site-packages/pandas/core/series.py", line 1040, in __getitem__
    return self._get_value(key)
  File "/data/Tra

In [ ]:
#Estratégico

In [45]:
# 4. Criando o insert para os medicamentos
cursor = conn.cursor()

for i, row in df_estrategico.iterrows():
    CO_CONSULTA = f"SELECT CO_SEQ_DROGA FROM C##DBMOBILE.TB_MEDSUS_DROGA WHERE NO_DROGA = '{row['DENOMINAÇÃO GENÉRICA']}'"
    cursor.execute(CO_CONSULTA)
    resultado = cursor.fetchone()

    if resultado is not None:
        resultado = int(resultado[0])

        # Verificar se a combinação de CO_DROGA e CO_CATEGORIA já existe
        query_check = f"SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CATEGORIA WHERE CO_DROGA = {resultado} AND CO_CATEGORIA = 2"
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        # Se a combinação não existir, inserir
        if count == 0:
            query = f"INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CATEGORIA (CO_SEQ_DROGA_CATEGORIA, CO_DROGA, CO_CATEGORIA, DT_CRIADO, ST_REGISTRO_ATIVO, CO_UTILIZADOR) VALUES( SQ_MEDDROGACAT_COSEQDROGACAT.NEXTVAL, {resultado}, 2, CURRENT_TIMESTAMP, 'S', 1541)"
            cursor.execute(query)
    else:
        # Obter o valor máximo de CO_SEQ_DROGA
        query_max = "SELECT MAX(CO_SEQ_DROGA) FROM C##DBMOBILE.TB_MEDSUS_DROGA"
        cursor.execute(query_max)
        max_value = cursor.fetchone()[0]

        # Adicionar 1 ao valor máximo
        new_value = max_value + 1

        # Usar o novo valor na consulta de inserção
        query = f"INSERT INTO C##DBMOBILE.TB_MEDSUS_DROGA (CO_SEQ_DROGA, NO_DROGA, DT_ATUALIZADO, DT_CRIADO, ST_REGISTRO_ATIVO, CO_UTILIZADOR, ST_PUBLICADO, DS_JUSTIFICATIVA) VALUES( {new_value}, '{row['DENOMINAÇÃO GENÉRICA']}', CURRENT_TIMESTAMP , CURRENT_TIMESTAMP , 'S' , 1541, 'S' , NULL )"
        cursor.execute(query)

        query = f"INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CATEGORIA (CO_SEQ_DROGA_CATEGORIA, CO_DROGA, CO_CATEGORIA, DT_CRIADO, ST_REGISTRO_ATIVO, CO_UTILIZADOR) VALUES( SQ_MEDDROGACAT_COSEQDROGACAT.NEXTVAL, {new_value}, 2, CURRENT_TIMESTAMP, 'S', 1541)"
        cursor.execute(query)

conn.commit()

In [47]:
# 6. Categoria campo
import math
import traceback
cursor = conn.cursor()

for i, row in df_estrategico.iterrows():
    try:
        CO_CONSULTA = f"SELECT CO_SEQ_DROGA FROM C##DBMOBILE.TB_MEDSUS_DROGA WHERE NO_DROGA = '{row['DENOMINAÇÃO GENÉRICA']}'"
        cursor.execute(CO_CONSULTA)
        resultado = cursor.fetchone()
        resultado = int(resultado[0])

        # FORMA FARMACEUTICA
        forma_farmaceutica = row['FORMA FARMACÊUTICA'].replace("'", "''")

        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 3"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0:
            query01 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 3, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 1, 'S', '{forma_farmaceutica}')"""
            cursor.execute(query01)

        #GRUPO
        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 4"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0 and not math.isnan(row['GRUPO']):
            query02 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 4, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 2, 'S', '{row['GRUPO']}')"""
            cursor.execute(query02)
        # if math.isnan(row['GRUPO']):
        #     query02 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 4, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 2, 'S', '{row['GRUPO']}')"""
        #     cursor.execute(query02)

        #CID ou DOENÇA
        if not math.isnan(row['CID ou DOENÇA']):
            lista_original07 = str(row['DOCUMENTOS NECESSÁRIOS']).split('\n')
            lista_alterada07 = ["<p>" + texto + "</p>" for texto in lista_original07]
            texto_final07 = ''.join(lista_alterada07)
            
            query03 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 5, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 3, 'S', '{texto_final07}')"""
            cursor.execute(query03)

        # Documentos Necessários
        lista_original = str(row['DOCUMENTOS NECESSÁRIOS']).split('\n')
        lista_alterada = ["<p>" + texto + "</p>" for texto in lista_original]
        texto_final = ''.join(lista_alterada)

        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 6"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0:
            query04 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 6, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 4, 'S', '{texto_final}')"""
            cursor.execute(query04)

        #COMO ACESSAR
        if not math.isnan(row['COMO ACESSAR']):
            lista_original01 = str(row['COMO ACESSAR']).split('\n')
            lista_alterada01 = ["<p>" + texto + "</p>" for texto in lista_original01]
            texto_final01 = ''.join(lista_alterada01)

            query05 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 7, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 5, 'S', '{texto_final01}')"""
            cursor.execute(query05)

        #ONDE ENCONTRAR
        if row['ONDE ENCONTRAR']:
            lista_original02 = str(row['ONDE ENCONTRAR']).split('\n')
            lista_alterada02 = ["<p>" + texto + "</p>" for texto in lista_original02]
            texto_final02 = ''.join(lista_alterada02)

            query06 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 8, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 6, 'S', '{texto_final02}')"""
            cursor.execute(query06)       

        conn.commit()
    except Exception as e:
        print("Ocorreu um erro: ", e)
        print(traceback.format_exc())
        conn.rollback()

Ocorreu um erro:  must be real number, not str
Traceback (most recent call last):
  File "/tmp/ipykernel_75060/1081429952.py", line 29, in <module>
    if count == 0 and not math.isnan(row['GRUPO']):
TypeError: must be real number, not str

Ocorreu um erro:  must be real number, not str
Traceback (most recent call last):
  File "/tmp/ipykernel_75060/1081429952.py", line 29, in <module>
    if count == 0 and not math.isnan(row['GRUPO']):
TypeError: must be real number, not str

Ocorreu um erro:  must be real number, not str
Traceback (most recent call last):
  File "/tmp/ipykernel_75060/1081429952.py", line 29, in <module>
    if count == 0 and not math.isnan(row['GRUPO']):
TypeError: must be real number, not str

Ocorreu um erro:  must be real number, not str
Traceback (most recent call last):
  File "/tmp/ipykernel_75060/1081429952.py", line 29, in <module>
    if count == 0 and not math.isnan(row['GRUPO']):
TypeError: must be real number, not str

Ocorreu um erro:  must be real numbe

In [ ]:
# Especializado

In [48]:
# 4. Criando o insert para os medicamentos
cursor = conn.cursor()

for i, row in df_especializado.iterrows():
    CO_CONSULTA = f"SELECT CO_SEQ_DROGA FROM C##DBMOBILE.TB_MEDSUS_DROGA WHERE NO_DROGA = '{row['DENOMINAÇÃO GENÉRICA']}'"
    cursor.execute(CO_CONSULTA)
    resultado = cursor.fetchone()

    if resultado is not None:
        resultado = int(resultado[0])

        # Verificar se a combinação de CO_DROGA e CO_CATEGORIA já existe
        query_check = f"SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CATEGORIA WHERE CO_DROGA = {resultado} AND CO_CATEGORIA = 3"
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        # Se a combinação não existir, inserir
        if count == 0:
            query = f"INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CATEGORIA (CO_SEQ_DROGA_CATEGORIA, CO_DROGA, CO_CATEGORIA, DT_CRIADO, ST_REGISTRO_ATIVO, CO_UTILIZADOR) VALUES( SQ_MEDDROGACAT_COSEQDROGACAT.NEXTVAL, {resultado}, 3, CURRENT_TIMESTAMP, 'S', 1541)"
            cursor.execute(query)
    else:
        # Obter o valor máximo de CO_SEQ_DROGA
        query_max = "SELECT MAX(CO_SEQ_DROGA) FROM C##DBMOBILE.TB_MEDSUS_DROGA"
        cursor.execute(query_max)
        max_value = cursor.fetchone()[0]

        # Adicionar 1 ao valor máximo
        new_value = max_value + 1

        # Usar o novo valor na consulta de inserção
        query = f"INSERT INTO C##DBMOBILE.TB_MEDSUS_DROGA (CO_SEQ_DROGA, NO_DROGA, DT_ATUALIZADO, DT_CRIADO, ST_REGISTRO_ATIVO, CO_UTILIZADOR, ST_PUBLICADO, DS_JUSTIFICATIVA) VALUES( {new_value}, '{row['DENOMINAÇÃO GENÉRICA']}', CURRENT_TIMESTAMP , CURRENT_TIMESTAMP , 'S' , 1541, 'S' , NULL )"
        cursor.execute(query)

        query = f"INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CATEGORIA (CO_SEQ_DROGA_CATEGORIA, CO_DROGA, CO_CATEGORIA, DT_CRIADO, ST_REGISTRO_ATIVO, CO_UTILIZADOR) VALUES( SQ_MEDDROGACAT_COSEQDROGACAT.NEXTVAL, {new_value}, 3, CURRENT_TIMESTAMP, 'S', 1541)"
        cursor.execute(query)

conn.commit()

In [49]:
# 6. Categoria campo
import math
cursor = conn.cursor()

for i, row in df_especializado.iterrows():
    try:
        CO_CONSULTA = f"SELECT CO_SEQ_DROGA FROM C##DBMOBILE.TB_MEDSUS_DROGA WHERE NO_DROGA = '{row['DENOMINAÇÃO GENÉRICA']}'"
        cursor.execute(CO_CONSULTA)
        resultado = cursor.fetchone()
        resultado = int(resultado[0])

        # FORMA FARMACEUTICA
        forma_farmaceutica = row['FORMA FARMACÊUTICA'].replace("'", "''")

        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 3"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0:
            query01 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 3, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 1, 'S', '{forma_farmaceutica}')"""
            cursor.execute(query01)

        #GRUPO
        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 4"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0 and not math.isnan(row['GRUPO']):
            query02 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 4, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 2, 'S', '{row['GRUPO']}')"""
            cursor.execute(query02)
        # if math.isnan(row['GRUPO']):
        #     query02 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 4, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 2, 'S', '{row['GRUPO']}')"""
        #     cursor.execute(query02)

        #CID ou DOENÇA
        if not math.isnan(row['CID ou DOENÇA']):
            lista_original07 = str(row['DOCUMENTOS NECESSÁRIOS']).split('\n')
            lista_alterada07 = ["<p>" + texto + "</p>" for texto in lista_original07]
            texto_final07 = ''.join(lista_alterada07)
            
            query03 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 5, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 3, 'S', '{texto_final07}')"""
            cursor.execute(query03)

        # Documentos Necessários
        lista_original = str(row['DOCUMENTOS NECESSÁRIOS']).split('\n')
        lista_alterada = ["<p>" + texto + "</p>" for texto in lista_original]
        texto_final = ''.join(lista_alterada)

        query_check = f"""SELECT COUNT(*) FROM C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO WHERE CO_DROGA = {resultado} AND CO_CAMPO = 6"""
        cursor.execute(query_check)
        count = cursor.fetchone()[0]

        if count == 0:
            query04 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 6, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 4, 'S', '{texto_final}')"""
            cursor.execute(query04)

        #COMO ACESSAR
        if not math.isnan(row['COMO ACESSAR']):
            lista_original01 = str(row['COMO ACESSAR']).split('\n')
            lista_alterada01 = ["<p>" + texto + "</p>" for texto in lista_original01]
            texto_final01 = ''.join(lista_alterada01)

            query05 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 7, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 5, 'S', '{texto_final01}')"""
            cursor.execute(query05)

        #ONDE ENCONTRAR
        if row['ONDE ENCONTRAR']:
            lista_original02 = str(row['ONDE ENCONTRAR']).split('\n')
            lista_alterada02 = ["<p>" + texto + "</p>" for texto in lista_original02]
            texto_final02 = ''.join(lista_alterada02)

            query06 = f"""INSERT INTO C##DBMOBILE.RL_MEDSUS_DROGA_CAMPO (CO_SEQ_DROGA_CAMPO, CO_DROGA, CO_CAMPO, DT_CRIADO, DT_ATUALIZADO, ST_OBRIGATORIO, CO_UTILIZADOR, ST_PUBLICADO, NU_ORDEM, ST_REGISTRO_ATIVO, DS_CAMPO_VALOR) VALUES(SQ_MEDDROGACAM_COSEQDROGACAM.NEXTVAL, {resultado}, 8, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, 'N', 1541, 'S', 6, 'S', '{texto_final02}')"""
            cursor.execute(query06)       

        conn.commit()
    except Exception as e:
        print("Ocorreu um erro: ", e)
        print(traceback.format_exc())
        conn.rollback()

Ocorreu um erro:  must be real number, not str
Traceback (most recent call last):
  File "/tmp/ipykernel_75060/1851237266.py", line 36, in <module>
    if not math.isnan(row['CID ou DOENÇA']):
TypeError: must be real number, not str

Ocorreu um erro:  must be real number, not str
Traceback (most recent call last):
  File "/tmp/ipykernel_75060/1851237266.py", line 36, in <module>
    if not math.isnan(row['CID ou DOENÇA']):
TypeError: must be real number, not str

Ocorreu um erro:  must be real number, not str
Traceback (most recent call last):
  File "/tmp/ipykernel_75060/1851237266.py", line 36, in <module>
    if not math.isnan(row['CID ou DOENÇA']):
TypeError: must be real number, not str

Ocorreu um erro:  must be real number, not str
Traceback (most recent call last):
  File "/tmp/ipykernel_75060/1851237266.py", line 36, in <module>
    if not math.isnan(row['CID ou DOENÇA']):
TypeError: must be real number, not str

Ocorreu um erro:  must be real number, not str
Traceback (most r